# for coraal folder

# working on cleaning data


To remove all component details within the "Content" column in such a way that we only have text. 
1. Drop rows that contain anything different text. (Pauses, Actions, Unintelligble Speech)
1. Remove punctuation marks from every row. (Removing ".", ",", "'", "?", etc.)
1. Always keep the corresponding "StTime" and "EnTime".
1. You can ommit/remove the "Spkr" "Line" or line columns.

In [16]:
import pandas as pd
import numpy as np
import torchaudio
import torch

In [7]:
transcript_dir = "../notebooks/coraal/transcript/text"

transcript_df = pd.read_csv(transcript_dir+"/ATL_se0_ag1_f_01_1.txt", delimiter="\t")
transcript_df.head()

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_se0_ag1_f_01,0.4436,"They talking about, don't send him to his daddy.",2.4068
1,2,ATL_se0_ag1_f_01,2.4068,(pause 0.28),2.6829
2,3,ATL_se0_ag1_f_01,2.6829,You just need to go file for child support.,4.9538
3,4,ATL_int_01,5.1142,[/Oh man/.],5.6125
4,5,ATL_se0_ag1_f_01,5.1488,[Bye.],5.5995


In [17]:
transcript_df

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_se0_ag1_f_01,0.4436,"They talking about, don't send him to his daddy.",2.4068
1,2,ATL_se0_ag1_f_01,2.4068,(pause 0.28),2.6829
2,3,ATL_se0_ag1_f_01,2.6829,You just need to go file for child support.,4.9538
3,4,ATL_int_01,5.1142,[/Oh man/.],5.6125
4,5,ATL_se0_ag1_f_01,5.1488,[Bye.],5.5995
...,...,...,...,...,...
1228,1229,ATL_se0_ag1_f_01,1855.6029,[Wanna] play?,1856.1984
1229,1230,ATL_int_01,1855.8262,[Hm.],1856.1798
1230,1231,ATL_int_01,1856.3473,Yeah let me- let me see it.,1857.6250
1231,1232,ATL_se0_ag1_f_01,1858.3942,Your phone dead.,1859.3929


---

# cleaing df of unwanted data on a copy df

# droped rows with [], ect

In [18]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# Create a copy of the original DataFrame to avoid corruption
cleaned_df = transcript_df.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, and < … >
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/', '', content)
    # Remove punctuation marks
    content = re.sub(r'[^\w\s]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df['CleanedContent'] = cleaned_df['Content'].apply(clean_content)

# Drop rows where "CleanedContent" is empty after cleaning
cleaned_df = cleaned_df[cleaned_df['CleanedContent'] != '']

# Keep only the "StTime", "EnTime", and "CleanedContent" columns
final_df = cleaned_df[['StTime', 'EnTime', 'CleanedContent']]

# Display the cleaned DataFrame
print(final_df.head())


   StTime  EnTime                                 CleanedContent
0  0.4436  2.4068  They talking about dont send him to his daddy
1  2.4068  2.6829                                      pause 028
2  2.6829  4.9538     You just need to go file for child support
5  6.0026  6.3944                                            Why
6  6.3944  7.1901                                      pause 080


# removed rows still containg pause

In [25]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
# cleaned_df = transcript_df.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, and < … >
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/', '', content)
    # Remove punctuation marks
    content = re.sub(r'[^\w\s]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df['CleanedContent'] = cleaned_df['Content'].apply(clean_content)

# Drop rows where "CleanedContent" is empty after cleaning
cleaned_df = cleaned_df[cleaned_df['CleanedContent'] != '']

# Remove rows containing the word "pause" in "CleanedContent"
cleaned_df = cleaned_df[~cleaned_df['CleanedContent'].str.contains(r'\bpause\b', case=False, na=False)]

# Keep only the "StTime", "EnTime", and "CleanedContent" columns
final_df = cleaned_df[['StTime', 'EnTime', 'CleanedContent']]

# Display the cleaned DataFrame
print(final_df.head())


   StTime  EnTime                                 CleanedContent
0  0.4436  2.4068  They talking about dont send him to his daddy
2  2.6829  4.9538     You just need to go file for child support
5  6.0026  6.3944                                            Why
7  7.1901  7.5398                                            Why
8  8.6665  9.7375                           Okay whats your name


# fixing index misorder after row deletion

In [27]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
# cleaned_df = transcript_df.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, and < … >
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/', '', content)
    # Remove punctuation marks
    content = re.sub(r'[^\w\s]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df['CleanedContent'] = cleaned_df['Content'].apply(clean_content)

# Drop rows where "CleanedContent" is empty after cleaning
cleaned_df = cleaned_df[cleaned_df['CleanedContent'] != '']

# Remove rows containing the word "pause" in "CleanedContent"
cleaned_df = cleaned_df[~cleaned_df['CleanedContent'].str.contains(r'\bpause\b', case=False, na=False)]

# Keep only the "StTime", "EnTime", and "CleanedContent" columns
final_df = cleaned_df[['StTime', 'EnTime', 'CleanedContent']]

# Reset the index of the cleaned DataFrame
final_df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df.head())


   StTime  EnTime                                 CleanedContent
0  0.4436  2.4068  They talking about dont send him to his daddy
1  2.6829  4.9538     You just need to go file for child support
2  6.0026  6.3944                                            Why
3  7.1901  7.5398                                            Why
4  8.6665  9.7375                           Okay whats your name


# cleeaned df keeping puncuations and droping rows of Special symbols

In [74]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
# cleaned_df = transcript_df.copy()

# Function to check for unwanted patterns in the "Content" column
def contains_unwanted_patterns(content):
    patterns = [r'\[.*?\]', r'\<.*?\>', r'\/.*?\/', r'\(.*?\)']
    for pattern in patterns:
        if re.search(pattern, content):
            return True
    return False

# Apply the function to filter rows with unwanted patterns
cleaned_df = cleaned_df[~cleaned_df['Content'].apply(contains_unwanted_patterns)]

# Remove rows where "Content" is empty after removing unwanted patterns
cleaned_df = cleaned_df[cleaned_df['Content'].str.strip() != '']

# Remove rows containing the word "pause" in "Content"
cleaned_df = cleaned_df[~cleaned_df['Content'].str.contains(r'\bpause\b', case=False, na=False)]

# Keep only the "StTime", "EnTime", and "Content" columns
final_df = cleaned_df[['StTime', 'EnTime', 'Content']]

# Reset the index of the cleaned DataFrame
final_df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df.head())


   StTime  EnTime                                           Content
0  0.4436  2.4068  They talking about, don't send him to his daddy.
1  2.6829  4.9538       You just need to go file for child support.
2  6.0026  6.3944                                              Why?
3  7.1901  7.5398                                              Why?
4  8.6665  9.7375                           Okay, what's your name?


---

# checking df of clean vs not

In [21]:
final_df['CleanedContent']

0           They talking about dont send him to his daddy
1                                               pause 028
2              You just need to go file for child support
5                                                     Why
6                                               pause 080
                              ...                        
1227                             the um Game Pigeon games
1228                                                 play
1230                            Yeah let me let me see it
1231                                      Your phone dead
1232    My phone all the way it And the charger right ...
Name: CleanedContent, Length: 1014, dtype: object

In [26]:
final_df['CleanedContent']

0           They talking about dont send him to his daddy
2              You just need to go file for child support
5                                                     Why
7                                                     Why
8                                    Okay whats your name
                              ...                        
1227                             the um Game Pigeon games
1228                                                 play
1230                            Yeah let me let me see it
1231                                      Your phone dead
1232    My phone all the way it And the charger right ...
Name: CleanedContent, Length: 669, dtype: object

In [28]:
final_df['CleanedContent']

0          They talking about dont send him to his daddy
1             You just need to go file for child support
2                                                    Why
3                                                    Why
4                                   Okay whats your name
                             ...                        
664                             the um Game Pigeon games
665                                                 play
666                            Yeah let me let me see it
667                                      Your phone dead
668    My phone all the way it And the charger right ...
Name: CleanedContent, Length: 669, dtype: object

In [31]:
final_df['CleanedContent'].size

669

In [32]:

transcript_df['Content'].size

1233

In [76]:
final_df['Content']

0       They talking about, don't send him to his daddy.
1            You just need to go file for child support.
2                                                   Why?
3                                                   Why?
4                                Okay, what's your name?
                             ...                        
488       Oh. What kind of games you play on your phone?
489                                                 Psh.
490                          Yeah let me- let me see it.
491                                     Your phone dead.
492    My phone all the way- it- And the charger righ...
Name: Content, Length: 493, dtype: object

# look at different samples for missed cleaned data

In [73]:
final_df['CleanedContent'].sample(10)


329    Aint gonna lie I was like oh man I forgot how ...
104                                         whos younger
93                                     No thats for real
573                       Yeah thats all I could say too
66                                  I have four brothers
437                                               Im not
314    you can ak you cant ask them about that Like t...
195                                               So  um
450                                    Nah that was real
57                                              You know
Name: CleanedContent, dtype: object

In [33]:
final_df['CleanedContent'].describe()

count      669
unique     601
top       Okay
freq        14
Name: CleanedContent, dtype: object

In [125]:
# df dropping rows containing special symbols from documentation and keeping puncuations 
# sampling to see if df was cleaned to desired parameters
final_df['Content'].sample(10)


465    never experienced. And then y'all bring like, ...
251    Really all of them joints the same for real fo...
134                                    with that friend?
90                                                   I'm
270                                             Okay, so
321                                         Alright bro.
272                                   seventy-two cents?
39                                              And, uh,
186                                        whoever else.
247                                                Yeah.
Name: Content, dtype: object

In [94]:
# size of new df dropping rows with special symbols and keeping puncuations
final_df['Content'].size

493

In [126]:
final_df

,StTime,EnTime,Content
0,0.4436,2.4068,"They talking about, don't send him to his daddy."
1,2.6829,4.9538,You just need to go file for child support.
2,6.0026,6.3944,Why?
3,7.1901,7.5398,Why?
4,8.6665,9.7375,"Okay, what's your name?"
...,...,...,...
488,1839.6002,1841.3867,Oh. What kind of games you play on your phone?
489,1841.9157,1842.7283,Psh.
490,1856.3473,1857.6250,Yeah let me- let me see it.
491,1858.3942,1859.3929,Your phone dead.


In [24]:
# error not used now just tests
final_df[final_df['CleanedContent'] == []].sample(10)


ValueError: ('Lengths must match to compare', (1014,), (0,))

---

# corral dataset

# now adding wave and audio paths to data frame and combining them with their respective transcripts

In [127]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from scipy.io import wavfile
from IPython.display import Audio
import torch
import torchaudio
import torchaudio.transforms as transforms


# getting path to all transcripts in corral folder

In [129]:
# Set the path to your directory
transcript_dir = '../notebooks/coraal/transcript/text'

# Create a list of file paths
transcript_paths = [os.path.join(transcript_dir, filename) for filename in os.listdir(transcript_dir) if filename.endswith('.txt')]
transcript_paths


['../notebooks/coraal/transcript/text/ATL_se0_ag2_f_02_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag2_m_02_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag2_f_01_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag2_m_03_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag2_m_01_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_05_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_03_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_f_01_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_f_03_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_01_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_04_2.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_04_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_f_02_1.txt',
 '../notebooks/coraal/transcript/text/ATL_se0_ag1_m_02_1.txt']

# getting paths to all .wav files in coraal

In [130]:
# Set the path to your directory
audio_directory = '../notebooks/coraal/audio/wav'

# Create a list of file paths
audio_paths = [os.path.join(audio_directory, filename) for filename in os.listdir(audio_directory) if filename.endswith('.wav')]
audio_paths


['../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag2_f_02_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag2_m_01_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag2_f_01_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag2_m_03_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_05_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_f_03_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_01_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_03_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_f_01_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_04_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_04_2.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_m_02_1.wav',
 '../notebooks/coraal/audio/wav/ATL_se0_ag1_f_02_1.wav']

In [131]:
def transcript_properties(filepath):
    df = pd.read_csv(filepath, delimiter="\t", index_col="Line")
    df['Transcript Path'] = filepath
    return df

# List to hold individual DataFrames
transcript_df = []

# Iterate through each file path, read the DataFrame, and append it to the list
for path in transcript_paths:
    transcript_df.append(transcript_properties(path))

# Concatenate all DataFrames in the list into a single DataFrame
transcript_df = pd.concat(transcript_df).reset_index()

# Display the combined DataFrame
transcript_df.head()


,Line,Spkr,StTime,Content,EnTime,Transcript Path
0,1,ATL_int_01,0.7526,Hey what's going on?,2.5113,../notebooks/coraal/transcript/text/ATL_se0_ag...
1,2,ATL_int_01,2.5113,(pause 0.63),3.1447,../notebooks/coraal/transcript/text/ATL_se0_ag...
2,3,ATL_int_01,3.1447,I'm here with,4.1659,../notebooks/coraal/transcript/text/ATL_se0_ag...
3,4,ATL_int_01,4.1659,(pause 0.92),5.0830,../notebooks/coraal/transcript/text/ATL_se0_ag...
4,5,ATL_int_01,5.0830,/RD-NAME-2/.,5.8536,../notebooks/coraal/transcript/text/ATL_se0_ag...


In [155]:
transcript_df.size

143022

---

# cleaning before transcript goes into process 

In [290]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
# cleaned_df2 = transcript_df.copy() # swap merged_df and transcript_df

# Function to check for unwanted patterns in the "Content" column
def contains_unwanted_patterns(content):
    patterns = [r'\[.*?\]', r'\<.*?\>', r'\/.*?\/', r'\(.*?\)']
    for pattern in patterns:
        if re.search(pattern, content):
            return True
    return False

# Apply the function to filter rows with unwanted patterns
transcript_df = transcript_df[~transcript_df['Content'].apply(contains_unwanted_patterns)]

# Remove rows where "Content" is empty after removing unwanted patterns
transcript_df = transcript_df[transcript_df['Content'].str.strip() != '']

# Remove rows containing the word "pause" in "Content"
#cleaned_df2 = cleaned_df2[~cleaned_df['Content'].str.contains(r'\bpause\b', case=False, na=False)] # commented bc dataframe got empty for merge_df above

# Keep only the "StTime", "EnTime", and "Content" columns
# final_df2 = cleaned_df2#[['StTime', 'EnTime', 'Content']] commented out bc wanted to keep coulmns made already

# Reset the index of the cleaned DataFrame
transcript_df.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(transcript_df.head())


   Line              Spkr   StTime               Content   EnTime  \
0     1        ATL_int_01   0.7526  Hey what's going on?   2.5113   
1     3        ATL_int_01   3.1447         I'm here with   4.1659   
2    11        ATL_int_01  10.5346            say hello.  11.2899   
3    12  ATL_se0_ag2_f_02  12.1424                Hello.  12.8462   
4    13        ATL_int_01  13.4581                 Okay,  13.9629   

                                     Transcript Path  
0  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
1  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
2  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
3  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
4  ../notebooks/coraal/transcript/text/ATL_se0_ag...  


In [291]:
transcript_df.size

59838

---

# return to sebastian code

In [292]:
def split_audio_to_tensors(audio_path, interval=30):
    waveform, sample_rate = torchaudio.load(audio_path)
    total_length = waveform.size(1) / sample_rate  # in seconds
    num_chunks = int(np.ceil(total_length / interval))
    
    tensors = []
    start_times = []
    end_times = []

    for i in range(num_chunks):
        start = int(i * interval * sample_rate)
        end = int(min((i + 1) * interval * sample_rate, waveform.size(1)))
        chunk_waveform = waveform[:, start:end]
        tensors.append(chunk_waveform)
        start_times.append(start / sample_rate)
        end_times.append(end / sample_rate)
    
    return tensors, start_times, end_times


In [293]:
def split_transcript(transcript_df, interval=30):
    transcript_df['StTime'] = pd.to_timedelta(transcript_df['StTime'], unit='s')
    transcript_df['EnTime'] = pd.to_timedelta(transcript_df['EnTime'], unit='s')
    
    new_rows = []
    current_content = []
    current_start_time = pd.to_timedelta(0, unit='s')
    current_end_time = pd.to_timedelta(interval, unit='s')
    
    for _, row in transcript_df.iterrows():
        start_time = row['StTime']
        end_time = row['EnTime']
        content = row['Content']
        
        if start_time >= current_end_time:
            # Save the current chunk
            new_rows.append({
                'Content': ' '.join(current_content),
                'Start Time': current_start_time.total_seconds(),
                'End Time': current_end_time.total_seconds()
            })
            # Reset for the next chunk
            current_content = []
            current_start_time = current_end_time
            current_end_time += pd.to_timedelta(interval, unit='s')
        
        current_content.append(content)
    
    # Add the last chunk
    if current_content:
        new_rows.append({
            'Content': ' '.join(current_content),
            'Start Time': current_start_time.total_seconds(),
            'End Time': current_end_time.total_seconds()
        })
    
    return pd.DataFrame(new_rows)


In [294]:
# Load a small subset of data
def load_small_subset(transcript_path, audio_path, num_rows=120):
    # Load transcript data
    transcript_df = pd.read_csv(transcript_path, delimiter='\t')
    
    # Select a subset of rows
    transcript_subset = transcript_df.head(num_rows).copy()

    # Load audio data
    waveform, sample_rate = torchaudio.load(audio_path)
    
    return transcript_subset, waveform, sample_rate


---

# back for testing cleaning data for desired results

# code changed for processing no punc and all caps

In [315]:
# Load a small subset of data for demonstration
transcript_path = '../notebooks/coraal/transcript/text/ATL_se0_ag2_m_02_1.txt'  # Replace with your transcript file path
audio_path = '../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02_1.wav'  # Replace with your audio file path

# Process data
transcript_df_subset, waveform, sample_rate = load_small_subset(transcript_path, audio_path, num_rows=120)
tensors, start_times, end_times = split_audio_to_tensors(audio_path, interval=30)

# Create DataFrame for audio chunks
audio_data_subset = pd.DataFrame({
    'Audio Tensor': tensors,
    'Start Time': start_times,
    'End Time': end_times,
    'Audio Path': [audio_path] * len(tensors)
})

###############
# samuel found that after this line is when the column gets turned into intervals of 30 secs and the trans_df is unchanged before this step
# only change to sebastian code

import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")
# marker for code change
# import pandas as pd
# import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# Create a copy of the original DataFrame to avoid corruption
cleaned_df6 = transcript_df_subset.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, < … >, and ( … )
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/|\(.*?\)', '', content)
    # Remove special symbols while keeping punctuation marks
    content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
    # Capitalize all text
    content = content.upper()
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df6['Content'] = cleaned_df6['Content'].apply(clean_content)

# Remove rows where "Content" is empty after removing unwanted patterns
cleaned_df6 = cleaned_df6[cleaned_df6['Content'].str.strip() != '']

# Keep the "StTime", "EnTime", and "CleanedContent" columns
final_df6 = cleaned_df6#[['StTime', 'EnTime', 'Content']]

# Reset the index of the cleaned DataFrame
final_df6.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df6.head())

############
# comment everything below to not get a df of 5 rows

# Process the transcript subset
final_df6 = split_transcript(final_df6, interval=30)
# transcript_df_subset                  transcript_df_subset subsituted with final_df6
# Merge DataFrames
audio_data_subset['Start Time'] = audio_data_subset['Start Time'].apply(lambda x: round(x, 2))
audio_data_subset['End Time'] = audio_data_subset['End Time'].apply(lambda x: round(x, 2))
final_df6['Start Time'] = final_df6['Start Time'].apply(lambda x: round(x, 2))
final_df6['End Time'] = final_df6['End Time'].apply(lambda x: round(x, 2))

merged_df = pd.merge_asof(final_df6.sort_values('Start Time'),
                            audio_data_subset.sort_values('Start Time'),
                            on='Start Time', direction='backward')

# Drop redundant columns
merged_df = merged_df.drop(columns=['End Time_y'])
merged_df = merged_df.rename(columns={'End Time_x': 'End Time'})

#Display the merged DataFrame
merged_df.head()


   Line              Spkr  StTime                    Content  EnTime
0     1        ATL_int_01  0.8713  CAN YOU TELL ME YOUR NAME  1.9540
1     2  ATL_se0_ag2_m_02  2.9491                         UH  3.8242
2     4  ATL_se0_ag2_m_02  5.1544                        THE  5.4122
3     6  ATL_se0_ag2_m_02  6.1185                I WAS NAMED  7.1703
4     8  ATL_se0_ag2_m_02  7.9334          BY THE GOVERNMENT  9.8462


,Content,Start Time,End Time,Audio Tensor,Audio Path
0,CAN YOU TELL ME YOUR NAME UH THE I WAS NAMED B...,0.0,30.0,"[[tensor(-6.1035e-05), tensor(-3.0518e-05), te...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
1,WHAT DO YOU IDENTIFY E ETHNICALLY ETHNICITYWIS...,30.0,60.0,"[[tensor(-0.0110), tensor(-0.0120), tensor(-0....",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
2,THATS WHERE I MAINLY WENT TO SCHOOL COOL COOL ...,60.0,90.0,"[[tensor(0.0081), tensor(0.0074), tensor(0.006...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
3,UM THINK I LIVED IN NEW YORK FOR A WHILE IN BR...,90.0,120.0,"[[tensor(0.0004), tensor(0.0005), tensor(0.000...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
4,AND OR SONGWRITER AND PRODUCER,120.0,150.0,"[[tensor(0.0158), tensor(0.0150), tensor(0.014...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...


# end of code for testing

---

In [304]:
merged_df['Content']

0    Can you tell me your name? the- I was named Ok...
1    what do you identify e- ethnically, ethnicity-...
2    That's where I mainly went to school, Cool, co...
3    Um, think- I lived in New York for a while in ...
4                      and or songwriter and producer.
Name: Content, dtype: object

In [309]:
merged_df

,Content,Start Time,End Time,Audio Tensor,Audio Path
0,Can you tell me your name? the- I was named Ok...,0.0,30.0,"[[tensor(-6.1035e-05), tensor(-3.0518e-05), te...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
1,"what do you identify e- ethnically, ethnicity-...",30.0,60.0,"[[tensor(-0.0110), tensor(-0.0120), tensor(-0....",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
2,"That's where I mainly went to school, Cool, co...",60.0,90.0,"[[tensor(0.0081), tensor(0.0074), tensor(0.006...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
3,"Um, think- I lived in New York for a while in ...",90.0,120.0,"[[tensor(0.0004), tensor(0.0005), tensor(0.000...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...
4,and or songwriter and producer.,120.0,150.0,"[[tensor(0.0158), tensor(0.0150), tensor(0.014...",../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...


In [307]:
final_df6

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_int_01,0.8713,Can you tell me your name?,1.9540
1,4,ATL_se0_ag2_m_02,5.1544,the-,5.4122
2,6,ATL_se0_ag2_m_02,6.1185,I was named,7.1703
3,9,ATL_int_01,10.3501,Okay.,10.6955
4,10,ATL_se0_ag2_m_02,10.7316,"Um, but I've renamed myself",12.6395
...,...,...,...,...,...
58,114,ATL_se0_ag2_m_02,114.6238,Yeah.,114.8764
59,115,ATL_int_01,116.0468,"Um,",116.7274
60,117,ATL_int_01,117.7468,occupation?,118.5460
61,118,ATL_se0_ag2_m_02,118.9894,"Uh, I'm a professional musician",120.9589


# adding in capitilzation and removing puncuations

In [314]:
final_df6

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_int_01,0.8713,CAN YOU TELL ME YOUR NAME,1.9540
1,2,ATL_se0_ag2_m_02,2.9491,UH,3.8242
2,4,ATL_se0_ag2_m_02,5.1544,THE,5.4122
3,6,ATL_se0_ag2_m_02,6.1185,I WAS NAMED,7.1703
4,8,ATL_se0_ag2_m_02,7.9334,BY THE GOVERNMENT,9.8462
...,...,...,...,...,...
62,114,ATL_se0_ag2_m_02,114.6238,YEAH,114.8764
63,115,ATL_int_01,116.0468,UM,116.7274
64,117,ATL_int_01,117.7468,OCCUPATION,118.5460
65,118,ATL_se0_ag2_m_02,118.9894,UH IM A PROFESSIONAL MUSICIAN,120.9589


---

In [308]:
transcript_df_subset

,Line,Spkr,StTime,Content,EnTime
0,1,ATL_int_01,0.8713,Can you tell me your name?,1.9540
1,2,ATL_se0_ag2_m_02,2.9491,"Uh, (breathy)",3.8242
2,3,ATL_se0_ag2_m_02,3.8242,(pause 1.33),5.1544
3,4,ATL_se0_ag2_m_02,5.1544,the-,5.4122
4,5,ATL_se0_ag2_m_02,5.4122,(pause 0.71),6.1185
...,...,...,...,...,...
115,116,ATL_int_01,116.7274,(pause 1.02),117.7468
116,117,ATL_int_01,117.7468,occupation?,118.5460
117,118,ATL_se0_ag2_m_02,118.9894,"Uh, I'm a professional musician",120.9589
118,119,ATL_se0_ag2_m_02,120.9589,(pause 0.51),121.4694


---

In [156]:
# Create a copy of the original DataFrame to avoid corruption
cleaned_df1 = merged_df.copy()


# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, and < … >
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/', '', content)
    # Remove punctuation marks
    content = re.sub(r'[^\w\s]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df1['CleanedContent'] = cleaned_df1['Content'].apply(clean_content)

# Drop rows where "CleanedContent" is empty after cleaning
cleaned_df1 = cleaned_df1[cleaned_df['CleanedContent'] != '']

# Remove rows containing the word "pause" in "CleanedContent"
cleaned_df1 = cleaned_df1[~cleaned_df1['CleanedContent'].str.contains(r'\bpause\b', case=False, na=False)]

# Keep only the "StTime", "EnTime", and "CleanedContent" columns
final_df1 = cleaned_df1[['StTime', 'EnTime', 'CleanedContent']]

# Display the cleaned DataFrame
print(final_df1.head())


/var/folders/_v/k355m06x52vd4ypck1wz5nzh0000gp/T/ipykernel_33946/1449465836.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_df1 = cleaned_df1[cleaned_df['CleanedContent'] != '']


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

# cleaned dataframe too much for merge so using transcript df

In [161]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
cleaned_df2 = transcript_df.copy() # swap merged_df and transcript_df

# Function to check for unwanted patterns in the "Content" column
def contains_unwanted_patterns(content):
    patterns = [r'\[.*?\]', r'\<.*?\>', r'\/.*?\/', r'\(.*?\)']
    for pattern in patterns:
        if re.search(pattern, content):
            return True
    return False

# Apply the function to filter rows with unwanted patterns
cleaned_df2 = cleaned_df2[~cleaned_df2['Content'].apply(contains_unwanted_patterns)]

# Remove rows where "Content" is empty after removing unwanted patterns
cleaned_df2 = cleaned_df2[cleaned_df2['Content'].str.strip() != '']

# Remove rows containing the word "pause" in "Content"
#cleaned_df2 = cleaned_df2[~cleaned_df['Content'].str.contains(r'\bpause\b', case=False, na=False)] # commented bc dataframe got empty for merge_df above

# Keep only the "StTime", "EnTime", and "Content" columns
final_df2 = cleaned_df2#[['StTime', 'EnTime', 'Content']] commented out bc wanted to keep coulmns made already

# Reset the index of the cleaned DataFrame
final_df2.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df2.head())


   Line              Spkr   StTime               Content   EnTime  \
0     1        ATL_int_01   0.7526  Hey what's going on?   2.5113   
1     3        ATL_int_01   3.1447         I'm here with   4.1659   
2    11        ATL_int_01  10.5346            say hello.  11.2899   
3    12  ATL_se0_ag2_f_02  12.1424                Hello.  12.8462   
4    13        ATL_int_01  13.4581                 Okay,  13.9629   

                                     Transcript Path  
0  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
1  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
2  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
3  ../notebooks/coraal/transcript/text/ATL_se0_ag...  
4  ../notebooks/coraal/transcript/text/ATL_se0_ag...  


In [162]:
final_df2

,Line,Spkr,StTime,Content,EnTime,Transcript Path
0,1,ATL_int_01,0.7526,Hey what's going on?,2.5113,../notebooks/coraal/transcript/text/ATL_se0_ag...
1,3,ATL_int_01,3.1447,I'm here with,4.1659,../notebooks/coraal/transcript/text/ATL_se0_ag...
2,11,ATL_int_01,10.5346,say hello.,11.2899,../notebooks/coraal/transcript/text/ATL_se0_ag...
3,12,ATL_se0_ag2_f_02,12.1424,Hello.,12.8462,../notebooks/coraal/transcript/text/ATL_se0_ag...
4,13,ATL_int_01,13.4581,"Okay,",13.9629,../notebooks/coraal/transcript/text/ATL_se0_ag...
...,...,...,...,...,...,...
9968,2198,ATL_int_01,2421.2731,everything. I'm excited,2422.9474,../notebooks/coraal/transcript/text/ATL_se0_ag...
9969,2200,ATL_int_01,2423.3031,about your future.,2424.3962,../notebooks/coraal/transcript/text/ATL_se0_ag...
9970,2202,ATL_int_01,2425.0819,Definitely gonna,2425.8604,../notebooks/coraal/transcript/text/ATL_se0_ag...
9971,2205,ATL_int_01,2429.0196,"and, um,",2429.6589,../notebooks/coraal/transcript/text/ATL_se0_ag...


In [289]:
final_df2['Content'].sample(10)


9087                                            one word,
9838                                                 that
9536                                   I love good music.
1290                                                i- i-
8356                      since the day she was born that
5177                                                Damn.
1185                              sports was the way out.
9689                      And they don't- they don't even
5838    Get rid of the school debts, or some shit like...
4880                 and just, this is how it planned out
Name: Content, dtype: object

# diffrent kind of cleaning

### still didnt change even though data was cleaned before hand. need to clean at the end of merge_df

In [160]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# Create a copy of the original DataFrame to avoid corruption
cleaned_df3 = merged_df.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, and < … >
    content = re.sub(r'\[.*?\]|\<.*?\>|\/.*?\/', '', content)
    # Remove punctuation marks
    content = re.sub(r'[^\w\s]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df3['CleanedContent'] = cleaned_df3['Content'].apply(clean_content)

# Drop rows where "CleanedContent" is empty after cleaning
cleaned_df3 = cleaned_df3[cleaned_df3['CleanedContent'] != '']

# Keep only the "StTime", "EnTime", and "CleanedContent" columns
final_df3 = cleaned_df3#[['StTime', 'EnTime', 'CleanedContent']]

# Reset the index of the cleaned DataFrame
final_df3.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df3.head())


                                             Content  Start Time  End Time  \
0  Can you tell me your name? Uh, (breathy) (paus...         0.0      30.0   
1  (pause 0.12) what do you identify e- (pause 0....        30.0      60.0   
2  (pause 0.31) That's where I mainly went to sch...        60.0      90.0   
3  Um, (pause 0.37) <ts> (pause 0.40) think- I li...        90.0     120.0   
4       (pause 0.51) and or songwriter and producer.       120.0     150.0   

                                        Audio Tensor  \
0  [[tensor(-6.1035e-05), tensor(-3.0518e-05), te...   
1  [[tensor(-0.0110), tensor(-0.0120), tensor(-0....   
2  [[tensor(0.0081), tensor(0.0074), tensor(0.006...   
3  [[tensor(0.0004), tensor(0.0005), tensor(0.000...   
4  [[tensor(0.0158), tensor(0.0150), tensor(0.014...   

                                          Audio Path  \
0  ../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...   
1  ../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...   
2  ../notebooks/coraal/aud

# 5th time trying

In [302]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# Create a copy of the original DataFrame to avoid corruption
cleaned_df5 = merged_df.copy()

# Function to clean the "Content" column
def clean_content(content):
    # Remove anything within [ … ], / … /, < … >, and ( … )
    content = re.sub(r'\[.*?\]', '', content)
    content = re.sub(r'\<.*?\>', '', content)
    content = re.sub(r'\/.*?\/', '', content)
    content = re.sub(r'\(.*?\)', '', content)
    # Remove special symbols while keeping punctuation marks
    content = re.sub(r'[^a-zA-Z0-9\s\.\,\?\!\;\:\'\"\-]', '', content)
    # Strip leading and trailing whitespaces
    content = content.strip()
    return content

# Apply the cleaning function to the "Content" column
cleaned_df5['CleanedContent'] = cleaned_df5['Content'].apply(clean_content)

# Keep the "StTime", "EnTime", and "CleanedContent" columns
final_df5 = cleaned_df5#[['StTime', 'EnTime', 'CleanedContent']]

# Reset the index of the cleaned DataFrame
final_df5.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df5.head())


                                             Content  Start Time  End Time  \
0  Can you tell me your name? Uh, (breathy) (paus...         0.0      30.0   
1  (pause 0.12) what do you identify e- (pause 0....        30.0      60.0   
2  (pause 0.31) That's where I mainly went to sch...        60.0      90.0   
3  Um, (pause 0.37) <ts> (pause 0.40) think- I li...        90.0     120.0   
4       (pause 0.51) and or songwriter and producer.       120.0     150.0   

                                        Audio Tensor  \
0  [[tensor(-6.1035e-05), tensor(-3.0518e-05), te...   
1  [[tensor(-0.0110), tensor(-0.0120), tensor(-0....   
2  [[tensor(0.0081), tensor(0.0074), tensor(0.006...   
3  [[tensor(0.0004), tensor(0.0005), tensor(0.000...   
4  [[tensor(0.0158), tensor(0.0150), tensor(0.014...   

                                          Audio Path  \
0  ../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...   
1  ../notebooks/coraal/audio/wav/ATL_se0_ag2_m_02...   
2  ../notebooks/coraal/aud

---

# still cleaned too much

In [297]:
import pandas as pd
import re

# # Load your original DataFrame
# transcript_dir = "../notebooks/coraal/transcript/text"
# transcript_df = pd.read_csv(transcript_dir + "/ATL_se0_ag1_f_01_1.txt", delimiter="\t")

# # Create a copy of the original DataFrame to avoid corruption
cleaned_df4 = merged_df.copy() # swap merged_df and transcript_df

# Function to check for unwanted patterns in the "Content" column
def contains_unwanted_patterns(content):
    patterns = [r'\[.*?\]', r'\<.*?\>', r'\/.*?\/', r'\(.*?\)']
    for pattern in patterns:
        if re.search(pattern, content):
            return True
    return False

# Apply the function to filter rows with unwanted patterns
cleaned_df4 = cleaned_df4[~cleaned_df4['Content'].apply(contains_unwanted_patterns)]

# Remove rows where "Content" is empty after removing unwanted patterns
cleaned_df4 = cleaned_df4[cleaned_df4['Content'].str.strip() != '']

# Remove rows containing the word "pause" in "Content"
#cleaned_df2 = cleaned_df2[~cleaned_df['Content'].str.contains(r'\bpause\b', case=False, na=False)] # commented bc dataframe got empty for merge_df above

# Keep only the "StTime", "EnTime", and "Content" columns
final_df4 = cleaned_df4#[['StTime', 'EnTime', 'Content']] commented out bc wanted to keep coulmns made already

# Reset the index of the cleaned DataFrame
final_df4.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(final_df4.head())


Empty DataFrame
Columns: [Content, Start Time, End Time, Audio Tensor, Audio Path]
Index: []


In [ ]:
merged_df

---

# to iterate through df

In [10]:
# Iterate through each row in our dataframe
for row in transcript_df.iterrows():
    print(row)
    break

(0, Line                                                      1
Spkr                                       ATL_se0_ag1_f_01
StTime                                               0.4436
Content    They talking about, don't send him to his daddy.
EnTime                                               2.4068
Name: 0, dtype: object)


In [15]:
# Iterates through columns
for data in transcript_df:
    print(data)
    break

Line


In [14]:
for tuples in transcript_df.itertuples():
    display(tuples)
    break

Pandas(Index=0, Line=1, Spkr='ATL_se0_ag1_f_01', StTime=0.4436, Content="They talking about, don't send him to his daddy.", EnTime=2.4068)